Pos = 0
Neg = 1

In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import numpy as np
import re
import logging
import time

In [2]:
def splitData(data):
    global train_index, test_index
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=2)
    iteration=0
    
    for train_index, test_index in skf.split(data,data["labels"]):
        iteration=iteration+1
        startTime=time.time()
        x_train, x_test, y_train, y_test=data.text.values[train_index],data.text.values[test_index],data.labels.values[train_index],data.labels.values[test_index]
        training_Df=pd.DataFrame({"labels":y_train,"text":x_train},columns=["text","labels"])
        test_Df=pd.DataFrame({"labels":y_test,"text":x_test},columns=["text","labels"])
        #configurateModel()
        #createModel()

        
        trainModel(training_Df)
        predictModel(test_Df,x_train,x_test,y_train,y_test,startTime,iteration)
        
       

In [3]:
def trainModel(training_Df):
    global model
    print("Training Model Now")
    print(training_Df)
    train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 4,
             "overwrite_output_dir":True,
             "train_batch_size": 32, 
             "eval_batch_size": 32,
             "use_multiprocessing":False,
             "use_multiprocessing_for_evaluation":False,
             "no_save":True} 
    
    
    #
    #'bert', 'deepset/gbert-base''electra', 'deepset/gelectra-base'
    #
    #deepset/gelectra-base
    #model_args=ClassificationArgs(num_train_epochs=1)
    #, cuda_device=0
    
    model=ClassificationModel('bert', 'nlptown/bert-base-multilingual-uncased-sentiment', num_labels=2, use_cuda=True, args=train_args,ignore_mismatched_sizes=True)
    #df2 = training_Df.head(20)
    #print(df2)
    
    
    model.train_model(training_Df)

In [4]:
def predictModel(test_Df,x_train,x_test,y_train,y_test,startTime,iteration):
    #df3=test_Df.head(10)
    #print(df3)
    print("-------------------------")
    def p_multiclass(labels, preds):
        #print("Labels: ", labels)
        #print("Preds: ", preds)
        return preds
    #result, model_outputs, wrong_predictions = model.eval_model(df3)
    result, model_outputs, wrong_predictions=model.eval_model(test_Df,acc=accuracy_score,precision=p_multiclass)
    

    print(result)
    print("----------------")
    
    #precision is pred labels, work around, cant get the preds otherwise
    getData(result["precision"],y_train,y_test,startTime,iteration)
    
    

 
    sentiment=np.argmax(model_outputs)
    resultList=[]
    pos=0
    neg=0
    #ERROR DOESNT GET RIGHT RESULT!!!
    for number in sentiment:
        if number >0.5:
            resultList.append(0)
            neg=neg+1
        if number <0.5:
            resultList.append(1)
            pos=pos+1
        if number ==0.5:
            print(error)
    #print(resultList)
    print("Neg:", neg)
    print("Pos:", pos)

In [5]:
def getData(test_set,y_train,y_test,startTime,iteration):
   
    #y_test=y_test[:10]
    corporaType=""
    shortcut=""
    name=""
    totalTextUnits=0
    totalTestUnits=0
    totalTrainUnits=0
    accuracy=0
    f1_macro=0
    precision_macro=0
    recall_macro=0
    f1_micro=0
    precision_micro=0
    recall_micro=0
    matrix=[]
    f1_binary=0
    precision_binary=0
    recall_binary=0

    splitString=re.split("_|/",file)
    #print(splitString)
    corporaType=splitString[8]
    shortcut=splitString[9]
    name=splitString[10]

    totalTime=time.time()-startTime

    totalTextUnits=len(data.index)
    totalTestUnits=len(test_set)
    totalTrainUnits=len(y_train)

    accuracy=accuracy_score(y_test, test_set)
    
    f1_binary=f1_score(y_test, test_set, average="binary",pos_label=0)
    precision_binary=precision_score(y_test, test_set, average="binary",pos_label=0)
    recall_binary=recall_score(y_test, test_set, average="binary",pos_label=0)
        
    matrix=confusion_matrix(y_test, test_set, labels=[1,0])
        
    matrixFlat=convertMatrix(matrix)
    target_names = [0,1]
    classificationReport=classification_report(y_test, test_set, target_names=target_names, output_dict=True)
    
    
    
    #print(matrixFlat)
    saveData(corporaType,shortcut,name,totalTime,totalTextUnits,totalTestUnits,totalTrainUnits,accuracy,f1_macro,precision_macro,recall_macro,f1_micro,precision_micro,recall_micro,matrix,f1_binary,precision_binary,recall_binary,y_test,test_set,startTime,iteration,matrixFlat,classificationReport)

In [6]:
def convertMatrix(matrix):
    global flatMatrix
    array=[]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            array.append(matrix[i][j])
    flatMatrix = np.array(array)
    return flatMatrix

In [7]:
def saveData(corporaType,shortcut,name,totalTime,totalTextUnits,totalTestUnits,totalTrainUnits,accuracy,f1_macro,precision_macro,recall_macro,f1_micro,precision_micro,recall_micro,matrix,f1_binary,precision_binary,recall_binary,y_test,test_set,startTime,iteration, matrixFlat,classificationReport):
    #print(matrixFlat)
    df_svm_data=pd.read_csv("MultiDataKFold.tsv", sep="\t")
    df_svm_data_full=pd.read_csv("MultiDataKFoldFull.tsv", sep="\t")

    #df=pd.DataFrame(columns=["Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
    #          "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
    #        "F1 Micro","F1 Binary","Matrix"])

    allData={"Iteration":iteration,"Shortcut":shortcut,"Name":name,"Type":corporaType,"Time":totalTime,"Total Length":totalTextUnits,"Training Set":totalTrainUnits,"Test Set":totalTestUnits,"Accuracy":accuracy,"Precision Macro":precision_macro,
             "Precision Micro":precision_micro,"Precision Binary":precision_binary,"Recall Macro":recall_macro,"Recall Micro":recall_micro,"Recall Binary":recall_binary,"F1 Macro":f1_macro,
            "F1 Micro":f1_micro,"F1 Binary":f1_binary,"Matrix":matrixFlat}

    allDataFull={"Iteration":iteration,"Shortcut":shortcut,"Name":name,"Type":corporaType,"Time":totalTime,"Total Length":totalTextUnits,"Training Set":totalTrainUnits,"Test Set":totalTestUnits,"Accuracy":accuracy,"Precision Macro":precision_macro,
             "Precision Micro":precision_micro,"Precision Binary":precision_binary,"Recall Macro":recall_macro,"Recall Micro":recall_micro,"Recall Binary":recall_binary,"F1 Macro":f1_macro,
            "F1 Micro":f1_micro,"F1 Binary":f1_binary,"Matrix":matrixFlat,"Train Set Full":y_test,"Test Set Full":test_set}

    
    reportDict=transformReport(classificationReport)
    allData.update(reportDict)
    allDataFull.update(reportDict)
    #print(allData)
    df_new_data=pd.DataFrame([allData])
    df_new_data_full=pd.DataFrame([allDataFull])

    finalData_svm=pd.concat([df_svm_data,df_new_data])
    finalData_svm_full=pd.concat([df_svm_data_full,df_new_data_full])
    
    finalData_svm=finalData_svm[["Iteration","Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
              "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
            "F1 Micro","F1 Binary","Matrix","0 precision","0 recall","0 f1-score",
            "0 support","1 precision","1 recall","1 f1-score","1 support","accuracy accuracy",
            "macro avg precision","macro avg recall","macro avg f1-score","macro avg support","weighted avg precision","weighted avg recall",
            "weighted avg f1-score","weighted avg support"]]

    finalData_svm_full=finalData_svm_full[["Iteration","Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
              "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
            "F1 Micro","F1 Binary","Matrix","Train Set Full","Test Set Full","0 precision","0 recall","0 f1-score",
            "0 support","1 precision","1 recall","1 f1-score","1 support","accuracy accuracy",
            "macro avg precision","macro avg recall","macro avg f1-score","macro avg support","weighted avg precision","weighted avg recall",
            "weighted avg f1-score","weighted avg support"]]
    
    finalData_svm.to_csv("MultiDataKFold.tsv", sep="\t",index=False)
    finalData_svm_full.to_csv("MultiDataKFoldFull.tsv", sep="\t",index=False)

In [8]:
def transformReport(classificationReport):
    print(classificationReport)
    newDict={}
    for key in classificationReport.keys():
        mainName=str(key)
        if type(classificationReport[key]) != dict:
            name=mainName+" "+key
            newDict[name]=classificationReport[key]
            #print(key, "->",classificationReport[key],"-",)
            
        if type(classificationReport[key]) == dict:
            for k in classificationReport[key].keys():
                name=mainName+" "+k
                newDict[name]=classificationReport[key][k]
                #print(k,"->",classificationReport[key][k])
    #print("--------------")
    print("DICT:",newDict)
    return newDict

"../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE04_filmstarts_all_Preprocessed_binary_Transformer.tsv",
                "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE05_amazonreviews_Preprocessed_binary_Transformer_balanced.tsv",
                "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE02_scare_Preprocessed_binary_Transformer_balanced.tsv"
                
                "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/LT02_speechLessing_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/MI01_mlsa_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/MI02_germeval_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/MI03_corpusRauh_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/NA01_gersen_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/NA02_gerom_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/NA03_ompc_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/RE01_usage_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/RE03_critics_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/SM01_sb10k_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/SM02_potts_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/SM03_multiSe_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/SM04_gertwittersent_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/SM05_ironycorpus_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/SM06_celeb_Preprocessed_ternary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/RE02_scare_Preprocessed_ternary_Transformer_balanced.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Ternary/RE04_filmstarts_all_Preprocessed_ternary_Transformer.tsv"

In [ ]:
filesBinaryTest=["../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT01_gnd_Preprocessed_binary_Transformer.tsv",
                  "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT02_speechLessing_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT03_historicplays_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/MI01_mlsa_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/MI02_germeval_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/MI03_corpusRauh_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/NA01_gersen_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/NA02_gerom_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/NA03_ompc_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE01_usage_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE03_critics_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM01_sb10k_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM02_potts_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM03_multiSe_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM04_gertwittersent_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM05_ironycorpus_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM06_celeb_Preprocessed_binary_Transformer.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE02_scare_Preprocessed_binary_Transformer_balanced.tsv",
                 "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE04_filmstarts_all_Preprocessed_binary_Transformer.tsv",
                "../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE05_amazonreviews_Preprocessed_binary_Transformer_balanced.tsv"]
        




#"../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT01_gnd_Preprocessed_binary_Transformer.tsv"
#../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM04_gertwittersent_Preprocessed_binary_Transformer.tsv
#NA01_gersen_Preprocessed_binary_Transformer.tsv
def main():
    global data, file
    for file in filesBinaryTest:
        print("Start for corpora: ",file)
        data=pd.read_csv(file, sep="\t")
        #updateFile(data)
        splitData(data)
        #print(data)
        print("Finish")

main()


Start for corpora:  ../../../Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT01_gnd_Preprocessed_binary_Transformer.tsv
Training Model Now
                                                  text  labels
0    Ohne Zwang bist Du erwachsen, ohne Zwang gebli...       0
1         Er reist ab, lät zweitausend Pfund im Stich!       1
2    Und beten heimlich zu Gott für die bedrängte K...       1
3    Eine neue Prüfung hatte ich auszustehen, da me...       1
4    Von unsrer groen, abermaligen Trübsal und letz...       1
..                                                 ...     ...
104    So ist das Geld schlielich in Achtung angelegt!       0
105  Untreue kann nur durch Untreue gerächt werden;...       1
106  Sie hatte sich da ganz geschickt benommen und ...       0
107  Weil er aber kein flüssiges Geld hatte, verpfä...       1
108  Die Wunde heilt sehr gut  Sie werden kaum eine...       0

[109 rows x 2 columns]


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/4 [00:00<?, ?it/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1641 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/1641 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/1641 [00:00<?, ?it/s]

file="../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT03_historicplays_Preprocessed_binary_Transformer.tsv"
data=pd.read_csv(file, sep="\t")
data.labels = data.labels.astype(int)

data.to_csv("LT03_historicplays_Preprocessed_binary_Transformer.tsv", sep="\t",index=False)

filesBinaryTest=["../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT01_gnd_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT02_speechLessing_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/LT03_historicplays_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/MI01_mlsa_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/MI02_germeval_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/MI03_corpusRauh_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/NA01_gersen_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/NA02_gerom_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/NA03_ompc_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE01_usage_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE03_critics_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM01_sb10k_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM02_potts_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM03_multiSe_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM04_gertwittersent_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM05_ironycorpus_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/SM06_celeb_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE02_scare_Preprocessed_binary_Transformer_balanced.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE04_filmstarts_all_Preprocessed_binary_Transformer.tsv",
                 "../shared/Corpora/Preprocessed_Transformers_Less_Preprocessing/Binary/RE05_amazonreviews_Preprocessed_binary_Transformer_balanced.tsv"]